In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from itertools import chain
from sklearn.model_selection import train_test_split
from scipy.stats import zscore
from sklearn import metrics

from scipy.stats import chisquare,chi2_contingency
from sklearn.preprocessing import Normalizer
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix

Using TensorFlow backend.


## 1.Read the dataset

In [3]:
bank_df = pd.read_csv('bank.csv')

In [4]:
bank_df.shape

(10000, 14)

In [5]:
bank_df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
bank_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             10000 non-null int64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [7]:
bank_df.isna().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [8]:
# There is no empty values in the dataset

In [9]:
bank_df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
RowNumber,10000.0,5.000500e+03,2886.895680,1.00,2500.75,5.000500e+03,7.500250e+03,10000.00
CustomerId,10000.0,1.569094e+07,71936.186123,15565701.00,15628528.25,1.569074e+07,1.575323e+07,15815690.00
CreditScore,10000.0,6.505288e+02,96.653299,350.00,584.00,6.520000e+02,7.180000e+02,850.00
Age,10000.0,3.892180e+01,10.487806,18.00,32.00,3.700000e+01,4.400000e+01,92.00
Tenure,10000.0,5.012800e+00,2.892174,0.00,3.00,5.000000e+00,7.000000e+00,10.00
Balance,10000.0,7.648589e+04,62397.405202,0.00,0.00,9.719854e+04,1.276442e+05,250898.09
NumOfProducts,10000.0,1.530200e+00,0.581654,1.00,1.00,1.000000e+00,2.000000e+00,4.00
HasCrCard,10000.0,7.055000e-01,0.455840,0.00,0.00,1.000000e+00,1.000000e+00,1.00
IsActiveMember,10000.0,5.151000e-01,0.499797,0.00,0.00,1.000000e+00,1.000000e+00,1.00
EstimatedSalary,10000.0,1.000902e+05,57510.492818,11.58,51002.11,1.001939e+05,1.493882e+05,199992.48


## 2.Drop the columns which are unique for all users like IDs (2.5 points)

In [10]:
# Dropping RowNumber, CustomerId, Surname columns as these are unique values
bank_df.drop(columns = ["RowNumber","CustomerId","Surname"], axis =1, inplace =True)

In [11]:
bank_df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


## 3.Distinguish the feature and target set (2.5 points)

In [12]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [13]:
labelencoder= LabelEncoder()

In [14]:
bank_df['Gender']=labelencoder.fit_transform(bank_df['Gender'])

In [15]:
bank_df["Geography"].unique()

array(['France', 'Spain', 'Germany'], dtype=object)

In [16]:
Geography =  pd.get_dummies(bank_df["Geography"])

In [17]:
bank_df = pd.concat([bank_df, Geography],axis=1)

In [18]:
bank_df.drop(columns = ["Geography"], inplace =True, axis = 1)

In [19]:
bank_df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,France,Germany,Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1,0,0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0,0,1
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1,0,0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1,0,0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0,0,1


In [20]:
X = bank_df.drop(columns = ["Exited"])
y = bank_df["Exited"]

## 4. Divide the data set into Train and test sets

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 15,stratify=y)

## 5.Normalize the train and test data (2.5 points)

In [22]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [23]:
X_train.shape

(7000, 12)

## 6. Initialize & build the model (10 points)

In [24]:
from keras.models import Sequential
from keras.layers import Dense

In [25]:
classifier_model = Sequential()

In [26]:
# Adding the input layer and the first hidden layer
classifier_model.add(Dense(output_dim = 6, init = 'uniform', activation = 'relu', input_dim = 12))
# Adding the second hidden layer
classifier_model.add(Dense(output_dim = 6, init = 'uniform', activation = 'relu'))
# Adding the output layer
classifier_model.add(Dense(output_dim = 1, init = 'uniform', activation = 'sigmoid'))

Instructions for updating:
Colocations handled automatically by placer.


C:\Users\varuraje.ORADEV\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=12, units=6, kernel_initializer="uniform")`
  
C:\Users\varuraje.ORADEV\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=6, kernel_initializer="uniform")`
  after removing the cwd from sys.path.
C:\Users\varuraje.ORADEV\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1, kernel_initializer="uniform")`
  


In [27]:
classifier_model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

## 7.Optimize the model (5 points)

In [28]:
classifier_model.fit(X_train, y_train, batch_size = 10, nb_epoch = 100)

Instructions for updating:
Use tf.cast instead.


C:\Users\varuraje.ORADEV\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Epoch 1/100
7000/7000 [==============================] - 2s 242us/step - loss: 0.4874 - acc: 0.7957
Epoch 2/100
7000/7000 [==============================] - 1s 161us/step - loss: 0.4273 - acc: 0.7963
Epoch 3/100
7000/7000 [==============================] - 1s 164us/step - loss: 0.4214 - acc: 0.7963
Epoch 4/100
7000/7000 [==============================] - 1s 168us/step - loss: 0.4173 - acc: 0.8117
Epoch 5/100
7000/7000 [==============================] - 1s 172us/step - loss: 0.4143 - acc: 0.8269
Epoch 6/100
7000/7000 [==============================] - 1s 175us/step - loss: 0.4118 - acc: 0.8286
Epoch 7/100
7000/7000 [==============================] - 1s 171us/step - loss: 0.4102 - acc: 0.8331
Epoch 8/100
7000/7000 [==============================] - 1s 200us/step - loss: 0.4088 - acc: 0.8309
Epoch 9/100
7000/7000 [==============================] - 1s 169us/step - loss: 0.4073 - acc: 0.8320
Epoch 10/100
7000/7000 [==============================] - 1s 182us/step - loss: 0.4056 - acc: 0.8333

7000/7000 [==============================] - 1s 164us/step - loss: 0.3920 - acc: 0.8359
Epoch 81/100
7000/7000 [==============================] - 1s 180us/step - loss: 0.3918 - acc: 0.8371
Epoch 82/100
7000/7000 [==============================] - 1s 194us/step - loss: 0.3920 - acc: 0.8390 1s - l
Epoch 83/100
7000/7000 [==============================] - 1s 180us/step - loss: 0.3921 - acc: 0.8380
Epoch 84/100
7000/7000 [==============================] - 1s 178us/step - loss: 0.3917 - acc: 0.8379
Epoch 85/100
7000/7000 [==============================] - 1s 184us/step - loss: 0.3918 - acc: 0.8370
Epoch 86/100
7000/7000 [==============================] - 1s 173us/step - loss: 0.3913 - acc: 0.8396
Epoch 87/100
7000/7000 [==============================] - 1s 163us/step - loss: 0.3916 - acc: 0.8376
Epoch 88/100
7000/7000 [==============================] - 1s 161us/step - loss: 0.3917 - acc: 0.8391
Epoch 89/100
7000/7000 [==============================] - 1s 183us/step - loss: 0.3911 - acc: 0.8

## 8.Predict the results using 0.5 as a threshold (5 points)

In [29]:
y_predict = classifier_model.predict(X_test)
y_predict = (y_predict > 0.5)

In [30]:
print(y_predict)

[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]


## 9. Print the Accuracy score and confusion matrix (2.5 points)

In [31]:
cm = confusion_matrix(y_test, y_predict)

In [32]:
print(cm)

[[2304   85]
 [ 420  191]]


In [33]:
metrics.accuracy_score(y_test,y_predict)

0.8316666666666667

In [34]:
cr=metrics.classification_report(y_test,y_predict)
print(cr)

             precision    recall  f1-score   support

          0       0.85      0.96      0.90      2389
          1       0.69      0.31      0.43       611

avg / total       0.81      0.83      0.81      3000

